In [8]:
#!/usr/bin/env python
# coding: utf-8


#!/usr/bin/env python
# coding: utf-8


import numpy as np
import time as T
from joblib import Parallel, delayed
from scipy.interpolate import griddata,interpn,RegularGridInterpolator
import math
import os
import datetime
import cv2
from ftplib import FTP
import rioxarray
import xarray as xr
import paramiko
from scp import SCPClient
import subprocess
import datetime as dt
import numba as nb

target ='/data01/people/liwei/Data/Sichuan/'

SZA_PATH = '/data01/GEO/INPUT/ANGLE/Solar_Zenith_Angle_u2/'
SAA_PATH = '/data01/GEO/INPUT/ANGLE/Solar_Azimuth_Angle_u2/'
VZA_PATH = '/data01/GEO/INPUT/ANGLE/Viewer_Zenith_Angle/AHI_VZA_10.dat'
VAA_PATH = '/data01/GEO/INPUT/ANGLE/Viewer_Azimuth_Angle/AHI_VAA_10.dat'

LUT_PATH = '/data01/GEO/INPUT/LUT/'
CAMS_PATH = '/data01/GEO/INPUT/ATMOSPHERE/'
DN_PATH = '/data01/GEO/INPUT/'
CAMS_AERO_PATH = '/data01/GEO/INPUT/AEROSOL_TYPE/'
AL_PATH = '/data01/GEO/INPUT/ELEVATION_GEO/AHI/MERIT_DEM_AHI_10km.dat'


res = 0.01
u_lat,d_lat = 60,-60
l_lon,r_lon = 85,205

row_AHI = 12000
col_AHI = 12000

AHI_lat = np.linspace(35 - res/2 ,25 + res/2,1000)
AHI_lon = np.linspace(97 + res/2 ,109 - res/2,1200)


In [4]:

   
class H8_data:

    def __init__(self , account , pw , band , band_number , date):
        self.account = account
        self.pw = pw
        self.band = band
        self.band_number = band_number
        self.date = date
    
    def get_path(self):
        return '/data01/GEO/ORGDATA/H8AHI/hmwr829gr.cr.chiba-u.ac.jp/gridded/FD/V20151105/' + self.date[0:6] + '/' + self.band.upper() + '/'

    def get_filename(self):
        return self.date + "." + self.band + "." + self.band_number + ".fld.geoss.bz2"
    
    def DN2TBB(self,data):
        LUT=np.loadtxt(DN_PATH + 'count2tbb_v102/' + self.band + "." + self.band_number)
        return LUT[data,1]
    
    def file_path(self):
        return self.get_path() + self.get_filename() 
                 
    def download_H8data(self):
        client = paramiko.SSHClient()
        client.load_system_host_keys()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.connect(hostname='10.4.123.47', port=22, username=self.account, password=self.pw)
        scp = SCPClient(client.get_transport())
        sftp = client.open_sftp()

        try :
            sftp.stat(self.file_path())

        except FileNotFoundError:
            client.close()
            print("File Not Found")
            return 'No data'

        else:
            scp.get(self.file_path(), folder_original+'/')
            p = subprocess.Popen('lbzip2 -d {}{}'.format(folder_original+'/',self.file_path()[-33:]),shell=True)
            p.communicate()
            client.close()
            print ('Himawari8/AHI data Processed Finish')
            return folder_original + self.get_filename()[:-4]
            
    def read_H8data(self):
        H8_file_path = self.download_H8data()
        if self.band == "vis":
            sr = 12000
        elif self.band == "ext":
            sr = 24000
        else:
            sr = 6000
        if H8_file_path != 'No data':
            fp = open(H8_file_path,'rb')
            data = np.frombuffer(fp.read(),dtype='>u2').reshape(sr,sr)
            fp.close()
            data = self.DN2TBB(data)
            data = data/100
            print("data reading finish")
            return data[2500:3500,1200:2400]
        else:
            return 'No data'
        
class LUT_interpolation:
    def __init__(self,LUT_PATH):
        self.LUT_PATH = LUT_PATH
    
    def LUT_interpolation(self):
        
        X1 = np.loadtxt(self.LUT_PATH + "01_band4.csv",delimiter=",").reshape(2,8,12,5,17,17,19)
        X2 = np.loadtxt(self.LUT_PATH + "02_band4.csv",delimiter=",").reshape(2,8,12,5,17,17,19)
        X3 = np.loadtxt(self.LUT_PATH + "03_band4.csv",delimiter=",").reshape(2,8,12,5,17,17,19)
        # return X1, X2, X3
        
        fn1 = RegularGridInterpolator((aero_type,water,AOT,al,sza,vza,raa),X1,bounds_error=False,fill_value=np.nan)
        fn2 = RegularGridInterpolator((aero_type,water,AOT,al,sza,vza,raa),X2,bounds_error=False,fill_value=np.nan)
        fn3 = RegularGridInterpolator((aero_type,water,AOT,al,sza,vza,raa),X3,bounds_error=False,fill_value=np.nan)
        return fn1,fn2,fn3 
    
class AHI_angle:
    def __init__(self,date):
        self.date = date        
    def read_angle_data(self):
        
        AHI_date = self.date[4:11]
        
        sza_file_name = 'AHI_SZA_2018{}5.dat'.format(AHI_date)
        saa_file_name = 'AHI_SAA_2018{}5.dat'.format(AHI_date)
        
        AHI_SZA_File = open(SZA_PATH + sza_file_name,'rb')
        AHI_SZA = np.frombuffer(AHI_SZA_File.read(),dtype='u2').reshape(3000,3000) / 100
        AHI_SZA_File.close()
        AHI_SZA=cv2.resize(np.array(AHI_SZA,dtype='float64'),(row_AHI,col_AHI),interpolation=cv2.INTER_NEAREST)
        
        
        AHI_SAA_File = open(SAA_PATH + saa_file_name,'rb')
        AHI_SAA = np.frombuffer(AHI_SAA_File.read(),dtype='u2').reshape(3000,3000) / 100
        AHI_SAA_File.close()
        AHI_SAA=cv2.resize(np.array(AHI_SAA,dtype='float64'),(row_AHI,col_AHI),interpolation=cv2.INTER_NEAREST)
        
        return AHI_SZA[2500:3500,1200:2400],AHI_SAA[2500:3500,1200:2400]

class CAMS_data:
    def __init__(self,Y,M,D,H,MI,lat,lon):
        self.YYYY = Y
        self.MM = M
        self.DD = D
        self.HH = H
        self.MIN = MI
        self.lon = lon
        self.lat = lat
        
    def read_CAMS(self):

        if int(self.HH) >= 21:
            d1 = dt.datetime(int(self.YYYY),int(self.MM),int(self.DD))
            d2 = dt.datetime(int(self.YYYY),int(self.MM),int(self.DD)) + dt.timedelta(days=1)
            
            ds1 = xr.open_dataset(CAMS_PATH + d1.strftime('%Y') + d1.strftime('%m') + d1.strftime('%d') + '.nc')
            ds2 = xr.open_dataset(CAMS_PATH + d2.strftime('%Y') + d2.strftime('%m') + d2.strftime('%d') + '.nc')
            ds = xr.merge([ds1, ds2]) 
            
        else:
             ds = xr.open_dataset(CAMS_PATH + self.YYYY + self.MM + self.DD + '.nc')
                
        dtime = dt.datetime(int(self.YYYY),int(self.MM),int(self.DD),int(self.HH),int(self.MIN)+5)    
        ds = ds.interp(time = dtime,method = 'linear')
        ds = ds.interp(longitude=self.lon,latitude=self.lat,method="nearest")
        
        OZ = ds['gtco3'].values        
        WV = ds['tcwv'].values        
        AOT550 = ds['aod550'].values
#         Atmosphere data Unit conversion
        WV = WV/10
        OZ = OZ*46.6975764


#         Processing water vapor and ozone max and min
        OZ[OZ>=max(ozone)] = max(ozone)-(1/10000)
        OZ[OZ<=min(ozone)] = min(ozone)+(1/10000)
        WV[WV>=max(water)] = max(water)-(1/10000)
        WV[WV<=min(water)] = min(water)+(1/10000)
        AOT550[AOT550>=max(AOT)] = max(AOT)-(1/10000)
        AOT550[AOT550<=min(AOT)] = min(AOT)+(1/10000)
        
        return np.array(OZ).reshape(1000,1200),np.array(WV).reshape(1000,1200),np.array(AOT550).reshape(1000,1200)

    def read_CAMS_AERO(self):
        
                
        if int(self.HH) >= 21:
            d1 = dt.datetime(int(self.YYYY),int(self.MM),int(self.DD))
            d2 = dt.datetime(int(self.YYYY),int(self.MM),int(self.DD)) + dt.timedelta(days=1)
            
            ds1 = xr.open_dataset(CAMS_AERO_PATH + d1.strftime('%Y') + d1.strftime('%m') + d1.strftime('%d') + '.nc')
            ds2 = xr.open_dataset(CAMS_AERO_PATH + d2.strftime('%Y') + d2.strftime('%m') + d2.strftime('%d') + '.nc')
            ds = xr.merge([ds1, ds2]) 
            
        else:
             ds = xr.open_dataset(CAMS_AERO_PATH + self.YYYY + self.MM + self.DD + '.nc')
                
        dtime = dt.datetime(int(self.YYYY),int(self.MM),int(self.DD),int(self.HH),int(self.MIN)+5)
        ds = ds.interp(time = dtime,method = 'linear')
        ds = ds.interp(longitude=self.lon,latitude=self.lat,method="nearest")
        
        bc = ds['bcaod550'].values
        du = ds['duaod550'].values
        om = ds['omaod550'].values
        ss = ds['ssaod550'].values
        su = ds['suaod550'].values

        DL_6S = np.array(du)
        SL_6S = np.array(su) + np.array(bc)
        OC_6S = np.array(ss)
        WS_6S = np.array(om)

        Total = DL_6S + SL_6S + OC_6S + WS_6S

        precent_DL_6S = DL_6S / Total
        precent_SL_6S = SL_6S / Total
        precent_OC_6S = OC_6S / Total
        precent_WS_6S = WS_6S / Total
        P = np.dstack((precent_DL_6S,precent_WS_6S,precent_OC_6S,precent_SL_6S))
        Aerosol_type = np.where(np.amax(P,axis = 2) == precent_OC_6S,1,0)
        
        return Aerosol_type
    
def H8_Process(ACCOUNT,PW,Band,Band_number,Date):
    data = H8_data(ACCOUNT,PW,Band,Band_number,Date).read_H8data()
    return data

def remove_original_file(path):
    os.system('rm -rf {}'.format(path))
    
    
def mkdir(path):
    folder = os.path.exists(path) 
    if not folder:
        os.makedirs(path)
        
def Time_split(time):
    YYYY = time.strftime('%Y')
    MM = time.strftime('%m')
    DD = time.strftime('%d')
    HH = time.strftime('%H')
    MIN = time.strftime('%M')
    date = YYYY + MM + DD + HH + MIN
    return YYYY,MM,DD,HH,MIN,date
        
        
def calculate_6s_band4(i):
    
    Aero_input = Aerosol_type[i][Ocean_mask[i]]
    WV_input = WV[i][Ocean_mask[i]]
    AOT550_input = AOT550[i][Ocean_mask[i]]
    RAA_input = RAA[i][Ocean_mask[i]]
    SZA_input = AHI_SZA[i][Ocean_mask[i]]
    VZA_input = AHI_VZA[i][Ocean_mask[i]]
    AL_input = AHI_AL[i][Ocean_mask[i]]
    AHI_data_input = AHI_data[i][Ocean_mask[i]]

    xi = np.array([Aero_input,WV_input,AOT550_input,AL_input,SZA_input,VZA_input,RAA_input])
    xi = xi.T
    xa = fn1(xi)
    xb = fn2(xi)
    xc = fn3(xi)
    y = xa*AHI_data_input-xb
    SR = y/(1+xc*y)
    return SR

@nb.jit()
def get_water_idx():
    Water_idx = []
    for i in range(1000):
        line_idx =[]
        for j in range(1200):
            if Ocean_mask[i,j] == True:
                line_idx.append(j)

        Water_idx.append(line_idx)
    return Water_idx

Ocean_mask_PATH = '/data01/GEO/INPUT/LAND_MASK/Landmask_10.dat'
    
Mask_File = open(Ocean_mask_PATH,'rb')
Ocean_mask = np.frombuffer(Mask_File.read(),dtype='u1').reshape(12000,12000)[2500:3500,1200:2400].astype('bool')
Mask_File.close()    
    
    
    
Water_idx = get_water_idx()
sza = np.linspace(0,80,17)
vza = np.linspace(0,80,17)
water = np.linspace(0,7,8)
ozone = np.linspace(0.2,0.4,5)
AOT = np.array([0.01,0.05,0.1,0.15,0.2,0.3,0.4,0.6,0.8,1.0,1.5,2.0])
raa = np.linspace(0,180,19)
al = np.linspace(0,8,5)
aero_type = np.array([0,1])

fn1,fn2,fn3 = LUT_interpolation(LUT_PATH).LUT_interpolation()


VZA_File = open(VZA_PATH,'rb')
AHI_VZA = np.frombuffer(VZA_File.read(),dtype='u2').reshape(12000,12000)[2500:3500,1200:2400] * 0.01
VZA_File.close()

VAA_File = open(VAA_PATH,'rb')
AHI_VAA = np.frombuffer(VAA_File.read(),dtype='u2').reshape(12000,12000)[2500:3500,1200:2400] * 0.01
VAA_File.close()

AL_File = open(AL_PATH,'rb')
AHI_AL = np.frombuffer(AL_File.read(),dtype='u2').reshape(12000,12000)[2500:3500,1200:2400] * 0.001
AL_File.close()

In [9]:

date_start = '2018-01-01 00:00'
date_end = '2018-01-31 09:00'
date_t = dt.timedelta(minutes=10)
date_s = dt.datetime.strptime(date_start, "%Y-%m-%d %H:%M")
date_e = dt.datetime.strptime(date_end, "%Y-%m-%d %H:%M")
date_time_now = date_s
while date_time_now <= date_e:
    start_time = T.time()
    date_dl_str = date_time_now.strftime("%Y-%m-%d %H:%M" )

    YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)    
    print("start processing {}".format(date))

    # make dir
    folder_original = target + date +'_original/'
    folder_AC = target+'{}/{}/{}/'.format(YYYY,MM,DD)
    mkdir(folder_original)
    mkdir(folder_AC)


    S_Read_AHI_time = T.time()
    # Download AHI
    AHI_data = H8_Process('liwei','liwei00','vis','03',date)
    if AHI_data == 'No data':
        date_time_now = date_time_now + date_t
        remove_original_file(folder_original)
        continue
        
    E_Read_AHI_time = T.time()
    # Solar angle
    print('Start reading Angle data')
    S_ANGLE_time = T.time()
    AHI_SZA,AHI_SAA = AHI_angle(date).read_angle_data()
    RAA = abs(AHI_SAA - AHI_VAA)
    RAA[RAA>180]=360-RAA[RAA>180]
    print('Angle data read finished')
    E_ANGLE_time = T.time()

    # Atmosphereic data
    print('Start reading Atmospheric data')
    S_ATMOS_time = T.time()

    OZ,WV,AOT550 = CAMS_data(YYYY,MM,DD,HH,MIN,AHI_lat,AHI_lon).read_CAMS()
    Aerosol_type = CAMS_data(YYYY,MM,DD,HH,MIN,AHI_lat,AHI_lon).read_CAMS_AERO()
    print('Atmospheric data read finished')
    E_ATMOS_time = T.time()


    # SR calculate
    print('Start Calculate SR')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=40)(delayed(calculate_6s_band4)(i) for i in range(1000))
    SR = np.full((1000,1200),np.nan)
    for i in range(1000):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print('Calculate SR finished')
    
    # Save file and remove download input data
    print('Start Save SR file ')
    S_SAVE_time = T.time()
    
    SR=np.array(SR).reshape(1000,1200)
    SR = np.around(SR,4) * 10000
    SR = np.where(SR>=32767,32767,SR)
    SR = np.where(SR<=-32768,-32768,SR)
    SR_file=open(folder_AC+'H08_{}{}{}_{}{}_B04_SR.dat'.format(YYYY,MM,DD,HH,MIN),'wb')
    
    SR.astype('int16').tofile(SR_file)
    SR_file.close()
    print('Save SR file finished')
    remove_original_file(folder_original)
    print("delete file finish")
    E_SAVE_time = T.time()
    end_time=T.time()
    TIME=end_time-start_time
    print('time: {:.1f} secs, {:.1f} mins,{:.1f} hours'.format(TIME,TIME/60,TIME/3600))


    date_time_now = date_time_now + date_t

start processing 201801010000
Himawari8/AHI data Processed Finish
data reading finish
Start reading Angle data
Angle data read finished
Start reading Atmospheric data


<ipython-input-9-8772202d95b8>:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if AHI_data == 'No data':


Atmospheric data read finished
Start Calculate SR
Calculate SR finished
Start Save SR file 
Save SR file finished
delete file finish
time: 8.6 secs, 0.1 mins,0.0 hours
start processing 201801010010
Himawari8/AHI data Processed Finish
data reading finish
Start reading Angle data
Angle data read finished
Start reading Atmospheric data


<ipython-input-9-8772202d95b8>:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if AHI_data == 'No data':


Atmospheric data read finished
Start Calculate SR
Calculate SR finished
Start Save SR file 
Save SR file finished
delete file finish
time: 7.8 secs, 0.1 mins,0.0 hours
start processing 201801010020
Himawari8/AHI data Processed Finish
data reading finish
Start reading Angle data
Angle data read finished
Start reading Atmospheric data


<ipython-input-9-8772202d95b8>:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if AHI_data == 'No data':


Atmospheric data read finished
Start Calculate SR
Calculate SR finished
Start Save SR file 
Save SR file finished
delete file finish
time: 6.6 secs, 0.1 mins,0.0 hours
start processing 201801010030


FileNotFoundError: [Errno 2] No such file or directory